In [1]:
# PYTHON NAITIVE
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import struct
import json
import os
import shutil

# ROS
from sensor_msgs.msg import CameraInfo, Image
from cv_bridge import CvBridge, CvBridgeError

# GQCNN
from autolab_core import YamlConfig, Logger
import autolab_core.utils as utils
# from gqcnn import get_gqcnn_model, get_gqcnn_trainer, utils as gqcnn_utils
from perception import (BinaryImage, CameraIntrinsics, ColorImage, DepthImage,
                        RgbdImage)
from visualization import Visualizer2D as vis
# from gqcnn.grasping import (RobustGraspingPolicy,
#                             CrossEntropyRobustGraspingPolicy, RgbdImageState,
#                             FullyConvolutionalGraspingPolicyParallelJaw,
#                             FullyConvolutionalGraspingPolicySuction)
# from gqcnn.utils import GripperMode

# GQCNN Networks
# from policy_mujoco import GQCNN
# from grasp_network import VPNET

import numpy as np
from matplotlib.patches import Rectangle

# CUSTOM PACKAGES
from baxter_robot import BaxterRobot
from custom_realsense_sensor import RealSenseSensor
from realrobot_utils import *
from greedy_agent import *

# FOR VISUALIZATION"
import mpld3
from mpld3 import plugins
%matplotlib inline
from IPython.display import clear_output

In [3]:
rospy.init_node("collecting_6dof_realrobot_grasping_data")
realsense = RealSenseSensor("826212070576",frame='realsense')
camera_1 = RealSenseSensor("023422073834",frame='realsense')
camera_2 = RealSenseSensor("025222072118",frame='realsense')
baxter_right_arm = BaxterRobot('right',ik_trials=10)
baxter_left_arm = BaxterRobot('left',ik_trials=10)

[INFO] [1613814500.628909]: Robot Enabled
[INFO] [1613814503.145556]: Robot Enabled


## Calibration

In [4]:
calibration_joint_angles = [{'right_e0': 0.19251458887961942,
 'right_e1': 0.6189612479117644,
 'right_s0': 0.6254806662602774,
 'right_s1': -0.9541360500647273,
 'right_w0': -0.19059711289476267,
 'right_w1': 2.063971150099824,
 'right_w2': -0.15378157398551273},
                            {'right_e0': 1.2742698569762068,
 'right_e1': 1.4520600494823506,
 'right_s0': -0.08169290290373594,
 'right_s1': -1.1523049914420582,
 'right_w0': -0.4056283485977753,
 'right_w1': 1.5620040784992788,
 'right_w2': 0.3882446109795593},
                           {'right_e0': 0.6860729073817513,
 'right_e1': 1.2060923944749065,
 'right_s0': -0.3746748074410123,
 'right_s1': -1.430053589506177,
 'right_w0': 0.16835439147042416,
 'right_w1': 1.8917818066596865,
 'right_w2': -0.7389952445637981},
                           {'right_e0': 0.19711653124327566,
 'right_e1': 0.9644904203829539,
 'right_s0': 0.17640779060682257,
 'right_s1': -1.1332283070503495,
 'right_w0': 0.18791264651596318,
 'right_w1': 1.8676216092504911,
 'right_w2': -0.5376602661538376}]


p_realsense_to_endpoints = []
quat_realsense_to_endpoints = []
bin_heights = []
for calibration_joint_angle in calibration_joint_angles:
    baxter_right_arm.move_to_joint_angles(calibration_joint_angle)
    for _ in range(3):
        try:
            T_realsense_to_endpoint, _, T_base_to_chess = robot_realsense_calibration(baxter_right_arm, realsense, CHECKER_COLS = 9, CHECKER_ROWS = 6, SQUARE_SIZE = 0.025, show_images=False)
        except:
            T_realsense_to_endpoint, _, T_base_to_chess = robot_realsense_calibration(baxter_right_arm, realsense, CHECKER_COLS = 9, CHECKER_ROWS = 6, SQUARE_SIZE = 0.025, show_images=True)
            
        R_realsense_to_endpoint, p_realsense_to_endpoint = get_Rp_from_T(T_realsense_to_endpoint)
        p_realsense_to_endpoints.append(p_realsense_to_endpoint)
        quat_realsense_to_endpoint = quaternion_from_matrix(rot3x3_to_4x4(R_realsense_to_endpoint))
        quat_realsense_to_endpoints.append(quat_realsense_to_endpoint)
        _, p_base_to_chess = get_Rp_from_T(T_base_to_chess)
        bin_heights.append(p_base_to_chess[2])

Moving the right arm to the pose...
Moving the right arm to the pose...
Moving the right arm to the pose...
Moving the right arm to the pose...


In [13]:
quat_realsense_to_endpoint = np.median(quat_realsense_to_endpoints, axis=0)
p_realsense_to_endpoint = np.median(p_realsense_to_endpoints, axis=0)
R_realsense_to_endpoint = quaternion_matrix(quat_realsense_to_endpoint)[:3,:3]
T_realsense_to_endpoint = form_T(R_realsense_to_endpoint, p_realsense_to_endpoint)
print(T_realsense_to_endpoint)

T_realsense_to_endpoint[0,3] -= 0.01
T_realsense_to_endpoint[1,3] += 0.025
T_realsense_to_endpoint[2,3] -= 0.03

bin_height = np.mean(bin_heights)
print(T_realsense_to_endpoint)
print(bin_height)

[[-0.03984073 -0.998733   -0.0307427   0.01483392]
 [ 0.99909574 -0.04027461  0.01362544  0.05759827]
 [-0.01484632 -0.03017205  0.99943446  0.12677811]
 [ 0.          0.          0.          1.        ]]
[[-0.03984073 -0.998733   -0.0307427   0.00483392]
 [ 0.99909574 -0.04027461  0.01362544  0.08259827]
 [-0.01484632 -0.03017205  0.99943446  0.09677811]
 [ 0.          0.          0.          1.        ]]
-0.17006824056096756


In [14]:
init_realsense_position = np.array([[0.65, -0.288, 0.65+bin_height]]).T
init_realsense_quaternion = np.array([ 0.70710678, -0.70710678, 0., 0.])

T_base_to_new_realsense = form_T(quaternion_matrix(init_realsense_quaternion)[:3,:3], init_realsense_position)
T_base_to_new_endpoint = T_base_to_new_realsense.dot(T_realsense_to_endpoint)

init_eef_position = T_base_to_new_endpoint[:3,3]
init_eef_quaternion = quaternion_from_matrix(T_base_to_new_endpoint)
init_eef_jointangles = baxter_right_arm.move_to_pose(init_eef_position, init_eef_quaternion)
current_eef_pose = baxter_right_arm._limb.endpoint_pose()
baxter_right_arm.gripper_open()

print(init_eef_position.T)
print(current_eef_pose['position'])
print(init_eef_quaternion)
print(current_eef_pose['orientation'])
print(init_eef_jointangles)

# raw_input("Remove the chessboard and place the bin. Are you done?")

Moving the right arm to the pose...
[ 0.56740173 -0.29283392  0.38315365]
Point(x=0.5674740118233019, y=-0.2906140216193931, z=0.3837165775151905)
[ 0.02003569  0.99965784  0.0152339  -0.00712038]
Quaternion(x=0.01872067021839483, y=0.9996534745378176, z=0.016991185101320836, w=-0.007332597077138134)
{'right_s0': -0.27729115716620595, 'right_s1': -1.3380035233003633, 'right_w0': -0.19874101401935695, 'right_w1': 1.6872687249320877, 'right_w2': 0.24728247862408442, 'right_e0': 1.2960948822826435, 'right_e1': 1.3940321166118148}


## Functions

In [15]:
def find_block_pos():
    print('########################### GO TO INITIAL ###########################################')
    init_eef_jointangles = baxter_right_arm.move_to_pose(init_eef_position, init_eef_quaternion)
    current_eef_pose = baxter_right_arm._limb.endpoint_pose()
    baxter_right_arm.gripper_open()

    print('########################### FIND A BLOCK ###########################################')
    K_realsense, D_realsense = realsense.get_realsense_camera_K_and_D()
    bin_input_rgb_im, bin_input_depth_im, _, _, bin_depth_im, _= get_input_rgbd_image(realsense, crop_size = 430, resize_width = 256, resize_height = 256)

    rgb_= copy.deepcopy(bin_input_rgb_im)
    _, edged = cv2.threshold(rgb_[:,:,0],20,255, cv2.THRESH_BINARY_INV)
    plt.imshow(edged)
    plt.show()

    center_x, center_y = None, None
    _, contours, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if 600 < w*h < 1200:
            print(w*h)
            cv2.rectangle(rgb_,(x,y),(x+w,y+h),(0,255,0),2)
            center_x = x + w//2
            center_y = y + h//2
            cv2.circle(rgb_, (center_x, center_y), 2, (255, 0, 0), 1)
    plt.imshow(rgb_)
    plt.show()

    if center_x is None or center_y is None:
        print('ERROR: Cannot find the block!!')

    print('########################### FIND BLOCK POS ###################################')
    endpoint_pose_base = baxter_right_arm._limb.endpoint_pose()
    T_base_to_endpoint = get_transformation_from_pose(endpoint_pose_base)
    T_endpoint_to_realsense = np.linalg.inv(T_realsense_to_endpoint)

    midx, midy = realsense._intrinsics[:2,2]

    pixel = np.array([center_x, center_y], np.int32)
    psi = 0.0
    grasp_depth = 0.68
    raw_pixel = inverse_raw_pixel(pixel, midx, midy, cs=430)
    diff_endpoint_position_realsense = inverse_projection(bin_depth_im, raw_pixel, K_realsense, D_realsense)
    diff_endpoint_position_realsense[2] = grasp_depth
    diff_endpoint_quaternion_realsense = quaternion_from_matrix(T_realsense_to_endpoint.dot(rotation_matrix(psi, [0.,0.,1.])))

    T_realsense_to_new_endpoint = form_T(quaternion_matrix(diff_endpoint_quaternion_realsense)[:3,:3], diff_endpoint_position_realsense)
    T_base_to_new_endpoint =T_base_to_endpoint.dot(T_endpoint_to_realsense.dot(T_realsense_to_new_endpoint))
    goal_eef_position = T_base_to_new_endpoint[:3,3]
    print('Goal EEF position', goal_eef_position)
    
    return goal_eef_position, center_x, center_y

def check_grasp():
    endeff_torque = baxter_right_arm._gripper.force()
    if endeff_torque > 25.:
        return True
    else:
        return False

In [16]:
def move_to_pixel(center_x, center_y):
    pixel = np.array([center_x, center_y], np.int32)
    grasp_depth = 0.70
    psi = 0.0
    midx, midy = realsense._intrinsics[:2,2]
    raw_pixel = inverse_raw_pixel(pixel, midx, midy, cs=430)
    
    endpoint_pose_base = baxter_right_arm._limb.endpoint_pose()
    T_base_to_endpoint = get_transformation_from_pose(endpoint_pose_base)
    T_endpoint_to_realsense = np.linalg.inv(T_realsense_to_endpoint)
    
    K_realsense, D_realsense = realsense.get_realsense_camera_K_and_D()
    bin_input_rgb_im, bin_input_depth_im, _, _, bin_depth_im, _= get_input_rgbd_image(realsense, crop_size = 430, resize_width = 256, resize_height = 256)
    
    diff_endpoint_position_realsense = inverse_projection(bin_depth_im, raw_pixel, K_realsense, D_realsense)
    diff_endpoint_position_realsense[2] = grasp_depth
    diff_endpoint_quaternion_realsense = quaternion_from_matrix(T_realsense_to_endpoint.dot(rotation_matrix(psi, [0.,0.,1.])))
    
    T_realsense_to_new_endpoint = form_T(quaternion_matrix(diff_endpoint_quaternion_realsense)[:3,:3], diff_endpoint_position_realsense)
    T_base_to_new_endpoint =T_base_to_endpoint.dot(T_endpoint_to_realsense.dot(T_realsense_to_new_endpoint))
    goal_eef_position = T_base_to_new_endpoint[:3,3]
    goal_eef_quaternion = quaternion_from_matrix(T_base_to_new_endpoint)
    goal_eef_jointangles = baxter_right_arm.move_to_pose(goal_eef_position+[0.0, 0.0, 0.05], goal_eef_quaternion, timeout=10.0)
    goal_eef_jointangles = baxter_right_arm.move_to_pose(goal_eef_position, goal_eef_quaternion, timeout=10.0)
    
    return goal_eef_position
    
def get_curr_eef_pos():
    endpoint_pose_base = baxter_right_arm._limb.endpoint_pose()
    T_base_to_endpoint = get_transformation_from_pose(endpoint_pose_base)
    return T_base_to_endpoint[:3, 3]

def rl_step(action, curr_grasp):
    eef_pos = get_curr_eef_pos()
    mov_dist = 0.03
    z_min = -0.23674
    grasp = curr_grasp
    if action<10:
        if action < 8:
            mov_degree = action * np.pi / 4.0
            eef_pos = eef_pos + np.array([mov_dist * np.cos(mov_degree), mov_dist * np.sin(mov_degree), 0.0])
        elif action == 8:
            eef_pos = eef_pos + np.array([0.0, 0.0, mov_dist])
        elif action == 9:
            eef_pos = eef_pos + np.array([0.0, 0.0, -mov_dist])
            if eef_pos[2] < z_min:
                eef_pos[2] = z_min
        init_eef_jointangles = baxter_right_arm.move_to_pose(eef_pos, init_eef_quaternion, timeout=3.0)
    else:
        if action == 10:
            baxter_right_arm.gripper_close()
            grasp = 1.0
        elif action == 11:
            baxter_right_arm.gripper_open()
            grasp = 0.0
    rospy.sleep(0.1)
    new_eef_pos = get_curr_eef_pos()
    return new_eef_pos, grasp

# Collecting Demonstrations

In [17]:
def get_state(cam_1, cam_2):
    rgb_1, depth_1, _, _, _, _= get_input_rgbd_image(cam_1, crop_size = 430, resize_width = 128, resize_height = 128)
    rgb_2, depth_2, _, _, _, _= get_input_rgbd_image(cam_2, crop_size = 430, resize_width = 128, resize_height = 128)
    state = np.zeros([128, 128, 8])
    state[:, :, :3] = rgb_1/255
    state[:, :, 3] = np.clip(depth_1, 0.0, 5.0)
    state[:, :, 4:7] = rgb_2/255
    state[:, :, 7] = np.clip(depth_2, 0.0, 5.0)
    return state

def record_state(buff_a, buff_s, buff_d, task):
    return

In [18]:
task = 'place' # 'reach', 'pick', 'place'
mov_dist = 0.03
agent = GreedyAgent(task=task, mov_dist=mov_dist)

In [ ]:
spawn_range = 0.0
baxter_right_arm.gripper_open()
goal_eef_position, center_x, center_y = find_block_pos()

if task=='reach':
    start_pos = init_eef_position.copy()
    start_pos[0] -= 0.15
    start_pos[2] = -0.20
    start_pos[:2] += spawn_range * np.random.uniform(low=-1.0, high=1.0, size=2)
    goal_pos = goal_eef_position
    baxter_right_arm.move_to_pose(start_pos, init_eef_quaternion, timeout=5.0)
    baxter_right_arm.gripper_close()
    curr_grasp = 1.0
elif task=='pick':
    start_pos = init_eef_position.copy()
    start_pos[0] -= 0.15
    start_pos[2] = -0.10
    start_pos[:2] += spawn_range * np.random.uniform(low=-1.0, high=1.0, size=2)
    goal_pos = goal_eef_position
    baxter_right_arm.move_to_pose(start_pos, init_eef_quaternion, timeout=5.0)
    baxter_right_arm.gripper_open()
    curr_grasp = 0.0
elif task=='place':
    start_pos = np.zeros(3)
    start_pos = move_to_pixel(center_x, center_y)
    start_z = start_pos[2]
    start_pos[2] = -0.10
    goal_pos = np.array([0.75, -0.30, start_z])
    baxter_right_arm.gripper_close()
    baxter_right_arm.move_to_pose(start_pos, init_eef_quaternion, timeout=5.0)
    curr_grasp = 1.0
    

In [ ]:
curr_pos = get_curr_eef_pos()
success = False

buff_action = []
buff_state = []
buff_done = []
while True:
    action = agent.get_action(curr_pos, goal_pos, curr_grasp)
    curr_pos, curr_grasp = rl_step(action, curr_grasp)
    state = get_state(camera_1, camera_2)
    print(action, curr_grasp)
    
    buff_action.append(action)
    buff_state.append(state)
    
    if task=='reach' and np.linalg.norm(curr_pos - goal_pos) < mov_dist/2:
        success = True
    if task=='pick' and curr_pos[2] > -0.18 and check_grasp():
        success = True
    if task=='place' and np.linalg.norm(curr_pos - goal_pos) < mov_dist/2:
        success = True
    if success:
        print('Success!!')
        buff_done.append(1)
        break
    else:
        buff_done.append(0)

Moving the right arm to the pose...


In [58]:
if success:
    record_state(buff_action, buff_state, buff_done, task)

if task=='reach':
    pass
elif task=='pick':
    place_pos = np.zeros(3)
    place_pos[:2] = curr_pos[:2]
    place_pos[2] = -0.20
    baxter_right_arm.move_to_pose(place_pos, init_eef_quaternion, timeout=5.0)
    baxter_right_arm.gripper_open()
elif task=='place':
    baxter_right_arm.gripper_open()
baxter_right_arm.move_to_pose(start_pos, init_eef_quaternion, timeout=5.0)

Moving the right arm to the pose...


{'right_e0': 1.2075328019027234,
 'right_e1': 1.7853791072502672,
 'right_s0': -0.21821528391596315,
 'right_s1': -0.20921187387599413,
 'right_w0': -1.4443574766555198,
 'right_w1': 1.187141909316366,
 'right_w2': 0.7249360606401783}

In [ ]:
DATA_PATH = "/home/dof6/catkin_ws/src/samsung_manipulation/scripts/brain_data"